# Open-source LLM's as Agents with `ChatHuggingFace`


Open source LLMs are becoming viable general purpose agents. The goal of this notebook is to demonstrate how to make use of open-source LLMs as chat models using [Hugging Face Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index) with [LangChain's `ChatHuggingFace`]() to enable their usage and experimentation with agent-based workflows.

In particular, we will:
1. Utilize the [HuggingFaceEndpoint](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_endpoint.py) (or [HuggingFaceTextGenInference](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_text_gen_inference.py) or [HuggingFaceHub](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/llms/huggingface_hub.py)) integration to call a [HF Inference Endpoint](https://huggingface.co/inference-endpoints) that's serving an LLM via [Text Generation Inference (TGI)](https://huggingface.co/docs/text-generation-inference/index)
2. Utilize the `ChatHuggingFace` class that interfaces between LangChain's [Chat Messages](https://python.langchain.com/docs/modules/model_io/chat/#messages) and the hosted LLM by leveraging [Hugging Face's Chat Templates](https://huggingface.co/docs/transformers/chat_templating) to power a `ChatAgent` pipeline.
4. Demonstrate how to use an open-source LLM in a zero-shot ReAct Agent workflow, along with an open-source [Prometheus](https://huggingface.co/papers/2310.08491) model to perform "LLM as a Judge"-style evaluations on that Agent's ouputs.
5. Understand how several different open-source LLM's perform as general purpose agents by running an asynchronous evaluation pipeline using Prometheus as the judge. 



> Note: To run this notebook, you'll need to have:
> - an LLM deployed via a Hugging Face Inference Endpoint (the LLM must have a `chat_template` defined in its `tokenizer_config.json`)
> - A Hugging Face Token with access to the deployed endpoint saved as an environment variable: `HUGGINGFACEHUB_API_TOKEN`
> - A SerpAPI key saved as an environment variable: `SERPAPI_API_KEY`


## Setup

In [2]:
!pip install -q --upgrade transformers langchain text-generation python-dotenv jinja2 langchainhub numexpr datasets tqdm openai sentencepiece protobuf matplotlib google-search-results


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# ruff: noqa: E402

from dotenv import load_dotenv
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
import plotly.express as px
from datasets import load_dataset, concatenate_datasets


from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.agents import load_tools
from langchain.tools.render import render_text_description_and_args
from langchain.chat_models import ChatOpenAI

from prompts import SYSTEM_PROMPT, HUMAN_PROMPT, EVALUATION_PROMPT_TEMPLATE
from evaluation import build_evaluator, evaluate_answers
from run_agents import run_full_tests
from agents import build_hf_agent, build_openai_agent

load_dotenv(override=True)

True

## 1. Instantiate an LLM

You'll need to have a running Inference Endpoint available.

#### `HuggingFaceHub`

In [5]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

/home/ubuntu/venv/ml2/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


#### `HuggingFaceEndpoint`

In [ ]:
from langchain.llms import HuggingFaceEndpoint

endpoint_url = (
    "https://j02macig8fbovqhr.us-east-1.aws.endpoints.huggingface.cloud"  # zephyr
)
endpoint_url = (
    "https://iw8z8uxlp03gvuxc.us-east-1.aws.endpoints.huggingface.cloud"  # mixtral
)
llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 488,
        "top_k": 50,
        "repetition_penalty": 1.03,
    },
)

## 2. Create a wrapper for `BaseChatModel` to apply chat templates

In [6]:
from langchain.schema import HumanMessage
from chat_wrapper import HuggingFaceChatWrapper

Instantiate the model and some messages to pass.

In [ ]:
messages = [
    HumanMessage(
        content="You're a helpful assistant. What happens when an unstoppable force meets an immovable object?"
    ),
]

chat_model = HuggingFaceChatWrapper(llm=llm)
chat_model.model_id

Call the model.

In [ ]:
res = chat_model.invoke(messages)
print(res.content)

## Tests

Here we'll test out our model as a zero-shot ReAct Agent. The example below is taken from [here](https://python.langchain.com/docs/modules/agents/agent_types/react#using-chat-models).

Configure the agent with a `react-json` style prompt and access to a search engine and calculator.

### Define tools

In [7]:
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm)
# Rename tools in the same format used by other tools
TOOLS[0].name = "search"
TOOLS[1].name = "calculator"

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            SYSTEM_PROMPT + "\nSo, here is my question:" + HUMAN_PROMPT
        ),
    ]
)
prompt = prompt.partial(
    tool_description=render_text_description_and_args(TOOLS),
    tool_names=", ".join([t.name for t in TOOLS]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=TOOLS,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=5,
)

NameError: name 'chat_model' is not defined

In [9]:
example = {
    "input": "What is the age of Leonardo DiCaprio's current girlfriend, raised to the power 0.43?"
}

out = agent_executor.invoke(example)

NameError: name 'agent_executor' is not defined

# Evaluation Experiment 

- Find a group of test questions that use a certain set of tools to solve (HotpotQA)
- Run several different models as agent to solve the test questions (OS and proprietary)
- Use a LLM as a judge
- Use OS models as a judge
- Report correlations

### Evaluation dataset

In [98]:
full_eval_dataset = load_dataset("A-Roucher/agents_small_benchmark", split="train")

# Run tests

In [101]:
agent_endpoints = {
    # "Zephyr-7b-beta": "https://ytjpei7t003tedav.us-east-1.aws.endpoints.huggingface.cloud",
    # 'https://i45e7q2do4r8mw5k.us-east-1.aws.endpoints.huggingface.cloud',  # notus-7b
    # "Mixtral-7x8b": "https://iw8z8uxlp03gvuxc.us-east-1.aws.endpoints.huggingface.cloud",
    # 'https://h0qwp3dx2iixajoh.us-east-1.aws.endpoints.huggingface.cloud', # llama2-7b
    # 'Yi-34B-Chat': 'https://wtgjpwu76xh7cv7t.us-east-1.aws.endpoints.huggingface.cloud',
    # 'Llama2-7b': 'https://pbg28xzbho42zp1t.us-east-1.aws.endpoints.huggingface.cloud',
    "SOLAR-10.7B": "https://tj7v24gjtvozke28.us-east-1.aws.endpoints.huggingface.cloud"
}

agents = {name: build_hf_agent(endpoint) for name, endpoint in agent_endpoints.items()}

# uncomment below to test GPT4 as an agent
agents["GPT4"] = build_openai_agent(model_id="gpt-4-1106-preview")
agents["GPT3.5"] = build_openai_agent(model_id="gpt-3.5-turbo-1106")
print(agents)
# run eval
await run_full_tests(dataset=full_eval_dataset, agents=agents)
print("Evaluation complete!")

WARNING! endpoint_url is not default parameter.
                    endpoint_url was transferred to model_kwargs.
                    Please confirm that endpoint_url is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


{'SOLAR-10.7B': AgentExecutor(verbose=True, agent=RunnableAgent(runnable={
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tool_description': "search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {'tool_input': {'type': 'string'}}\ncalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}", 'tool_names': 'search, calculator'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_description', 'tool_names'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tool_description}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the to

  0%|          | 0/90 [00:00<?, ?it/s]

not found


not found


...

Yes, both Edward Laemmle and Pedro Costa are film directors, but they come from very different eras and backgrounds.

Edward Laemmle was an American film director active during the silent film era. He was a nephew of Carl Laemmle, the founder of Universal Pictures. Edward Laemmle directed numerous films in the 1920s but is not as widely known today as some of his contemporaries.

Pedro Costa is a contemporary Portuguese film director known for his work in the art-house genre. His films are noted for their careful visual composition, slow pacing, and the use of non-professional actors. He has gained a significant critical following for his unique approach to filmmaking and his focus on the lives of marginalized individuals. Some of his notable works include "Ossos" (1997), "In Vanda's Room" (2000), and "Vitalina Varela" (2019).

> Finished chain.
Result: {'agent_name': 'GPT4', 'agent_model_id': 'openai-chat', 'question': 'Are Edward Laemmle and Pedro Costa bkth film directors?', 'gt_ans

Yes, Grizzly Bear and Residual Kid are both associated with rock music, but they represent different styles and eras within the genre.

Grizzly Bear is an American rock band formed in Brooklyn, New York, in 2002. They are known for their blend of folk-rock, psychedelic pop, and experimental music. They have received critical acclaim for their complex harmonies, intricate melodies, and layered sounds.

Residual Kid is a rock band that started as a trio of young musicians from Austin, Texas. They are known for their grunge and punk-influenced sound. Despite their young ages, they have gained attention for their energetic performances and mature musicality. Residual Kid represents a newer generation of rock musicians, and they have been active since the early 2010s.

> Finished chain.
Result: {'agent_name': 'GPT4', 'agent_model_id': 'openai-chat', 'question': 'Are Grizzly Bear and Residual Kid both rock bands?', 'gt_answer': 'yes', 'prediction': 'Yes, Grizzly Bear and Residual Kid are bot

Yes, both Utah State University (USU) and Western Michigan University (WMU) are public universities in the United States.

Utah State University is a public land-grant research university in Logan, Utah. It is accredited by the Northwest Commission on Colleges and Universities.

Western Michigan University is a public research university in Kalamazoo, Michigan. It is also accredited by the Higher Learning Commission. Both universities offer a wide range of undergraduate and graduate programs.

> Finished chain.
Result: {'agent_name': 'GPT4', 'agent_model_id': 'openai-chat', 'question': 'Are Utah State University and Western Michigan University both public universities?', 'gt_answer': 'yes', 'prediction': 'Yes, both Utah State University (USU) and Western Michigan University (WMU) are public universities in the United States.\n\nUtah State University is a public land-grant research university in Logan, Utah. It is accredited by the Northwest Commission on Colleges and Universities.\n\nW

100%|██████████| 90/90 [06:52<00:00,  4.58s/it]

Based on the search results, Rachel Talalay is identified as a University of British Columbia film professor, indicating that she has had a teaching career in a university setting. There is no information found in the search results regarding W. S. Van Dyke's involvement in university teaching. Therefore, it can be concluded that Rachel Talalay has taught more university classes than W. S. Van Dyke.

> Finished chain.
Result: {'agent_name': 'GPT4', 'agent_model_id': 'openai-chat', 'question': 'Which American director has taught more University classes, Rachel Talalay or W. S. Van Dyke?', 'gt_answer': 'Rachel Talalay', 'prediction': "Based on the search results, Rachel Talalay is identified as a University of British Columbia film professor, indicating that she has had a teaching career in a university setting. There is no information found in the search results regarding W. S. Van Dyke's involvement in university teaching. Therefore, it can be concluded that Rachel Talalay has taught m

# Evaluate with LLM-as-a-judge

In [29]:
eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

# prometheus_endpoint = 'https://xe7njj8b9w43222g.us-east-1.aws.endpoints.huggingface.cloud'
# eval_chat_model = build_evaluator(prometheus_endpoint)
# eval_model_name = "Prometheus-13B-v1.0"

In [30]:
eval_chat_model.invoke("Hello!")

AIMessage(content='Hello! How can I assist you today? If you have any questions or need information on a topic, feel free to ask.')

In [31]:
for file in tqdm(glob.glob("output/*.json")):
    evaluate_answers(
        file,
        eval_chat_model,
        eval_model_name,
        EVALUATION_PROMPT_TEMPLATE,
    )

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

## Visualize results

In [57]:
results = []
for file in glob.glob("output/*.json"):
    print(file)
    with open(file) as f:
        results.append(pd.read_json(f))

result_df = pd.concat(results)
result_df["tools_used"] = result_df["intermediate_steps"].apply(
    lambda row: ([step["tool"] for step in row] if row is not None else None)
)
result_df["number_of_distinct_tools_used"] = result_df["tools_used"].apply(
    lambda row: (len(list(set(row))) if row is not None else None)
)
result_df["number_of_steps"] = result_df["tools_used"].apply(
    lambda row: (len(row) if row is not None else None)
)
result_df["no_prediction"] = result_df["prediction"].apply(
    lambda x: True if x is None else False
)
result_df.head(5)

output/SOLAR-10.7B-with_wiki.json
output/SOLAR-10.7B.json
output/GPT3.5-with_wiki.json
output/GPT4.json
output/Mixtral-7x8b.json
output/GPT3.5.json
output/Mixtral-7x8b-with_wiki.json


,agent_name,agent_model_id,question,gt_answer,prediction,intermediate_steps,parsing_error,iteration_limit_exceeded,agent_error,start_time,end_time,task,eval_score_GPT4,eval_feedback_GPT4,tools_used,number_of_distinct_tools_used,number_of_steps,no_prediction
0,SOLAR-10.7B-with_wiki,upstage/solar-10.7b-instruct-v1.0-style,Bob is tilling a plot of his garden. The plot ...,If Bob goes along the side that's 120 feet lon...,It will take Bob 3 minutes and 43.24 seconds (...,"[{'tool': 'calculator', 'tool_input': '110 * 1...",False,False,None,2023-12-18 10:40:54,2023-12-18 10:41:30,GSM8K,1,Feedback: The response provided is completely ...,"[calculator, calculator, calculator, calculator]",1.0,4.0,False
1,SOLAR-10.7B-with_wiki,upstage/solar-10.7b-instruct-v1.0-style,Earl has $90; Fred has $48; Greg has $36. Earl...,Earl will have $90 - $28 = $<<90-28=62>>62 aft...,None,None,False,False,ValueError('unknown format from LLM: Find the ...,2023-12-18 10:41:30,2023-12-18 10:41:41,GSM8K,1,Feedback: The response is completely incorrect...,None,NaN,NaN,True
2,SOLAR-10.7B-with_wiki,upstage/solar-10.7b-instruct-v1.0-style,A milk teas shop was able to sell a total of 5...,A milk tea shop sold 50 x 2/5 = <<50*2/5=20>>2...,15,"[{'tool': 'calculator', 'tool_input': '2/5 * 5...",False,False,None,2023-12-18 10:41:41,2023-12-18 10:42:07,GSM8K,5,Feedback: The response provided is completely ...,"[calculator, calculator, calculator]",1.0,3.0,False
3,SOLAR-10.7B-with_wiki,upstage/solar-10.7b-instruct-v1.0-style,A party venue has 4 tables that seat 6 people ...,Four 6-seater tables can accommodate 4 x 6 = <...,The total capacity of all the tables at the pa...,"[{'tool': 'calculator', 'tool_input': '4 * 6',...",False,False,None,2023-12-18 10:42:07,2023-12-18 10:42:36,GSM8K,5,Feedback: The response provided is completely ...,"[calculator, calculator, calculator, calculator]",1.0,4.0,False
4,SOLAR-10.7B-with_wiki,upstage/solar-10.7b-instruct-v1.0-style,Paul is collecting license plates from differe...,The proportion of plates that he has out of to...,None,None,False,False,ValidationError(model='search_wikipediaSchemaS...,2023-12-18 10:42:36,2023-12-18 10:42:51,GSM8K,1,Feedback: Since there is no response provided ...,None,NaN,NaN,True


In [59]:
def interpret_result(x):
    try:
        return int(x) - 1
    except:
        return 0


result_df[f"eval_score_{eval_model_name}"] = result_df[
    f"eval_score_{eval_model_name}"
].apply(interpret_result)

result_df.groupby("agent_name").agg(
    {
        f"eval_score_{eval_model_name}": "mean",
        "parsing_error": "sum",
        "iteration_limit_exceeded": "sum",
        "no_prediction": "sum",
    }
)

,eval_score_GPT4,parsing_error,iteration_limit_exceeded,no_prediction
agent_name,,,,
GPT3.5,2.510204,0,1,0
GPT3.5-with_wiki,2.142857,0,2,1
GPT4,2.857143,0,2,0
Mixtral-7x8b,2.285714,7,4,3
Mixtral-7x8b-with_wiki,1.571429,3,2,16
SOLAR-10.7B,2.040816,21,5,1
SOLAR-10.7B-with_wiki,2.163265,10,3,7


In [60]:
task_order = {
    "GAIA": 1,
    "GSM8K": 2,
    "HotpotQA-easy": 3,
    "HotpotQA-medium": 4,
    "HotpotQA-hard": 5,
}
result_df = (
    result_df.assign(x=result_df["task"].map(task_order))
    .sort_values("x")
    .drop("x", axis=1)
)
agg_df = result_df.groupby(["agent_name", "task"], sort=False)[
    f"eval_score_{eval_model_name}"
].mean()
display(agg_df.unstack())

task,GAIA,GSM8K,HotpotQA-easy,HotpotQA-medium,HotpotQA-hard
agent_name,,,,,
SOLAR-10.7B,0.2,2.866667,2.375,2.000,2.500
GPT3.5-with_wiki,0.8,2.400000,2.750,2.625,2.250
Mixtral-7x8b-with_wiki,0.3,1.066667,2.250,2.500,2.500
SOLAR-10.7B-with_wiki,1.2,2.266667,2.625,2.375,2.500
GPT3.5,1.2,2.933333,2.625,2.750,3.000
Mixtral-7x8b,0.7,2.533333,2.375,3.000,3.000
GPT4,1.2,3.733333,3.000,2.625,3.375


In [61]:
agg_df = agg_df / 4 * 100  # set results in percentage

In [62]:
fig = px.bar(
    agg_df.reset_index(),
    x="agent_name",
    y=f"eval_score_{eval_model_name}",
    color="task",
    text=f"eval_score_{eval_model_name}",
    title=f"Average Evaluation Score provided by {eval_model_name}",
    labels={
        "agent_name": "Agent",
        f"eval_score_{eval_model_name}": "Evaluation score",
        "task": "Task",
    },
)
fig.update_layout(width=1000, barmode="group", yaxis_range=[0, 100])
fig.update_traces(texttemplate="%{y:.0f}", textposition="inside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

### Study intermediate steps

In [98]:
result_df["has_answer"] = result_df["prediction"].apply(lambda row: (row is not None))
result_df_answers_only = result_df.loc[result_df["has_answer"] == True]
result_df_answers_only["correct_answer"] = (
    result_df_answers_only[f"eval_score_{eval_model_name}"] >= 3
)
aggregated_resuts = result_df_answers_only.groupby(
    ["agent_name", "task", "correct_answer"]
).agg({"number_of_steps": "mean"})

/tmp/ipykernel_24913/3739459843.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [99]:
px.bar(
    aggregated_resuts.reset_index(),
    x="task",
    y="number_of_steps",
    facet_col="agent_name",
    color="correct_answer",
    title="Mean Number of Steps to Solve a Task",
    barmode="group",
    labels={
        "agent_name": "Agent",
        "number_of_steps": "Mean Number of Steps",
        "correct_answer": "Answer correctness",
        "task": "Task",
    },
    width=1500,
)

#### visualizations to make
- avg model score by question difficulty
- count of parsing errors and iteration limit errors

## To-do:
- Generalize evaluation code
- Make evals fully async
- Select 3-5 models to evaluate on
- Run eval
- Write blog post
- Get LangChain PR merged

## Notes

- One of the main challenges with open LLMs is ensuring they adhere to the proper markdown JSON output!
- Another is when they serve as the evaluator, they struggle to handle the default "labeled_criteria" prompt and end up hallucinating. Needs to be modified / simplified.
- I also found that GPT4 actually did pretty poor at judging correctness given the default LangChain prompt!